Recall that the decomposition group of $f$ modulo $p$ is defined if $f$ has no repeated factors over the finite field $GF(p)$. This is checked by verifying that the highest common factor of $f$ and its formal derivative $f'$ is $1$.

The structure of the decomposition group, specifically the cycle type of its generator, corresponds to the degrees of the irreducible factors of $f$ when factored over $GF(p)$.

This is an efficient algorithm to find the factors of a polynomial based on their degrees. The polynomial $X^{p^r} - X$ is the product of all monic irreducible polynomials over $GF(p)$ whose degrees divide $r$. By taking the HCF of our target polynomial $f$ with $X^{p^r} - X$ for increasing r, we can progressively find all its irreducible factors.

In [80]:
import numbers
from collections import Counter
from tabulate import tabulate

class Polynomial:
    '''
    A class to represent polynomials with coefficients in a finite field GF(p).
    '''
    def __init__(self, coefficients, p):
        if not isinstance(p, int) or p <= 1:
            raise ValueError("p must be a prime integer greater than 1.")
        self.p = p

        if isinstance(coefficients, list):
            self.coeffs = {len(coefficients) - 1 - i: c % self.p
                           for i, c in enumerate(coefficients) if c % self.p != 0}
        elif isinstance(coefficients, dict):
            self.coeffs = {deg: c % self.p for deg, c in coefficients.items() if c % self.p != 0}
        else:
            raise TypeError("Coefficients must be a list or a dictionary.")

        if not self.coeffs: self.coeffs = {0: 0}

    def degree(self):
        if self.coeffs == {0: 0}: return -1
        return max(self.coeffs.keys())

    def __str__(self):
        if self.degree() == -1: return "0"
        parts = []
        for deg in sorted(self.coeffs.keys(), reverse=True):
            coeff = self.coeffs[deg]
            if coeff == 1 and deg != 0: coeff_str = ""
            else: coeff_str = str(coeff)
            if deg > 1: var_str = f"x^{deg}"
            elif deg == 1: var_str = "x"
            else: var_str = ""
            if deg != 0 and coeff != 1: parts.append(f"{coeff_str}{var_str}")
            elif deg != 0 and coeff == 1: parts.append(var_str)
            else: parts.append(coeff_str)
        return " + ".join(parts)

    def __repr__(self):
        return f"Polynomial({self.coeffs}, p={self.p})"

    def __eq__(self, other): return self.coeffs == other.coeffs and self.p == other.p

    def __add__(self, other):
        new_coeffs = self.coeffs.copy()
        for deg, coeff in other.coeffs.items():
            new_coeffs[deg] = (new_coeffs.get(deg, 0) + coeff) % self.p
        return Polynomial(new_coeffs, self.p)

    def __sub__(self, other):
        new_coeffs = self.coeffs.copy()
        for deg, coeff in other.coeffs.items():
            new_coeffs[deg] = (new_coeffs.get(deg, 0) - coeff) % self.p
        return Polynomial(new_coeffs, self.p)

    def __mul__(self, other):
        if isinstance(other, numbers.Number):
            return Polynomial({d: (c * other) % self.p for d, c in self.coeffs.items()}, self.p)
        new_coeffs = {}
        for d1, c1 in self.coeffs.items():
            for d2, c2 in other.coeffs.items():
                nd, nc = d1 + d2, (c1 * c2) % self.p
                new_coeffs[nd] = (new_coeffs.get(nd, 0) + nc) % self.p
        return Polynomial(new_coeffs, self.p)

    def __divmod__(self, divisor):
        if divisor.degree() == -1: raise ZeroDivisionError("Division by zero polynomial.")
        rem, quot = Polynomial(self.coeffs, self.p), Polynomial({}, self.p)
        div_deg, div_lc = divisor.degree(), divisor.coeffs[divisor.degree()]
        inv_lc = pow(div_lc, -1, self.p)
        while rem.degree() >= div_deg:
            rem_deg, rem_lc = rem.degree(), rem.coeffs[rem.degree()]
            t_deg, t_coeff = rem_deg - div_deg, (rem_lc * inv_lc) % self.p
            quot.coeffs[t_deg] = t_coeff
            term = Polynomial({t_deg: t_coeff}, self.p)
            rem -= term * divisor
        return quot, rem

    def __mod__(self, other): return divmod(self, other)[1]

    def derivative(self):
        '''
        Computes the formal derivative of the polynomial.
        '''
        new_coeffs = {}
        for deg, coeff in self.coeffs.items():
            if deg > 0:
                new_coeffs[deg - 1] = (coeff * deg) % self.p
        return Polynomial(new_coeffs, self.p)


In [81]:
def polynomial_hcf(a, b):
    while b.degree() != -1: a, b = b, a % b
    if a.degree() != -1:
        lc_inv = pow(a.coeffs[a.degree()], -1, a.p)
        a *= lc_inv
    return a

def polynomial_power(base, exp, modulus):
    res = Polynomial([1], base.p)
    base %= modulus
    while exp > 0:
        if exp % 2 == 1: res = (res * base) % modulus
        exp //= 2
        base = (base * base) % modulus
    return res

def sieve_of_eratosthenes(limit):
    '''
    Generates prime numbers up to a given limit.
    '''
    primes = []
    is_prime = [True] * (limit + 1)
    is_prime[0] = is_prime[1] = False
    for p in range(2, limit + 1):
        if is_prime[p]:
            primes.append(p)
            for multiple in range(p * p, limit + 1, p):
                is_prime[multiple] = False
    return primes

In [82]:
def compute_decomposition_group(f_int_coeffs, p):
    '''
    Computes the decomposition group of a polynomial f with integer coefficients modulo p.
    Returns the cycle structure as a sorted list of factor degrees.
    '''
    # Reduce polynomial f modulo p
    f = Polynomial(f_int_coeffs, p)

    # Check if the group is defined: HCF(f, f') must be 1
    f_prime = f.derivative()
    hcf = polynomial_hcf(f, f_prime)
    if hcf.degree() > 0:
        return "Undefined (f has repeated factors mod p)"

    # Decompose f into factors fr using distinct degree factorization
    cycle_structure = []
    f_to_factor = f
    x = Polynomial([1, 0], p)

    # We only need to check for factors up to half the degree of the remaining polynomial
    r = 1
    while f_to_factor.degree() > 2 * (r -1):
        # Compute h = X^(p^r) - X mod f_to_factor
        x_power_pr = polynomial_power(x, p**r, f_to_factor)
        h = x_power_pr - x

        # Find the product of all irreducible factors of degree r
        factor_product = polynomial_hcf(h, f_to_factor)

        if factor_product.degree() > 0:
            # We found a factor product. It might contain multiple factors of degree r.
            # (A full equal-degree factorization step would be needed to separate
            # them, but for the cycle structure, just their degrees are sufficient).
            num_factors = factor_product.degree() // r
            cycle_structure.extend([r] * num_factors)
            f_to_factor, _ = divmod(f_to_factor, factor_product)
        r += 1

    # If there's a remaining factor, it must be irreducible
    if f_to_factor.degree() > 0:
        cycle_structure.append(f_to_factor.degree())

    return sorted(cycle_structure)

polynomials_data = {
    "x^2+x+41": [1, 1, 41],
    "x^3+2x+1": [1, 0, 2, 1],
    "x^3+x^2-2x-1": [1, 1, -2, -1],
    "x^4-2x^2+4": [1, 0, -2, 0, 4],
    "x^4-x^3-4x+16": [1, -1, 0, -4, 16],
    "x^4-2x^3+5x+5": [1, -2, 0, 5, 5],
    "x^4+7x^2+6x+7": [1, 0, 7, 6, 7],
    "x^4+3x^3-6x^2-9x+7": [1, 3, -6, -9, 7],

    "x^5+36": [1, 0, 0, 0, 0, 36],
    "x^5-5x+3": [1, 0, 0, -5, 3],
    "x^5+x^3-3x^2+3": [1, 0, 1, -3, 0, 3],
    "x^5-11x^3+22x-11": [1, 0, -11, 0, 22, -11],
    "x^6+x+1": [1, 0, 0, 0, 0, 1, 1],
    "x^7-2x^6+2x+2": [1, -2, 0, 0, 0, 2, 2],
    "x^7+x^4-2x^2+8x+4": [1, 0, 0, 1, 0, -2, 8, 4],
    "x^7+x^5-4x^4-x^3+5x+1": [1, 0, 1, -4, -1, 5, 1, 1]
}

def format_cycle_structure(decomposition):
    '''
    Format the cycle structure string.
    '''
    if isinstance(decomposition, list):
        counts = Counter(decomposition)
        return " ".join([f"{d}" if c == 1 else f"{d}^{c}" for d, c in sorted(counts.items())])
    return "Undef." # For undefined groups

def run_analysis_and_tabulate(polynomials, limit=97):
    '''
    Runs the decomposition analysis for a list of polynomials and prints a formatted table.
    '''
    primes = sieve_of_eratosthenes(limit)
    headers = ["p"] + list(polynomials.keys())
    table_data = []

    for p in primes:
        row = [p]
        for poly_coeffs in polynomials.values():
            result = compute_decomposition_group(poly_coeffs, p)
            row.append(format_cycle_structure(result))
        table_data.append(row)

    num_poly_cols = 8 # Number of polynomial columns per table chunk
    num_chunks = (len(headers) - 1 + num_poly_cols - 1) // num_poly_cols
    for i in range(num_chunks):
        start_col = i * num_poly_cols + 1
        end_col = start_col + num_poly_cols
        chunk_headers = [headers[0]] + headers[start_col:end_col]
        chunk_data = [[row[0]] + row[start_col:end_col] for row in table_data]
        print(f"\nDecomposition Groups (Part {i+1})")
        print(tabulate(chunk_data, headers=chunk_headers, stralign="center"))


run_analysis_and_tabulate(polynomials_data)


Decomposition Groups (Part 1)
  p   x^2+x+41    x^3+2x+1    x^3+x^2-2x-1    x^4-2x^2+4    x^4-x^3-4x+16    x^4-2x^3+5x+5    x^4+7x^2+6x+7    x^4+3x^3-6x^2-9x+7
---  ----------  ----------  --------------  ------------  ---------------  ---------------  ---------------  --------------------
  2      2          1 2            3            Undef.         Undef.              4             Undef.              Undef.
  3      2           3             3            Undef.         Undef.           Undef.           Undef.               2^2
  5      2           3             3             2^2              4             Undef.             2^2               Undef.
  7      2           3           Undef.          2^2              4                4               1^4               1^2 2
 11      2          1 2            3             2^2           Undef.              4               2^2               1^2 2
 13      2          1 2           1^3            2^2             2^2              1 3       

The Chebotarev density theorem states that the Galois group $G(f)$ of a polynomial $f(x)$ acts as a group of permutations on the roots of $f(x)$. Each element in $G(f)$ has a specific cycle structure so when $f(x)$ is factored modulo $p$ (where $p$ does not divide the discriminant of $f$), the degrees of its irreducible factors correspond to the cycle structure of a Frobenius element in the Galois group. The density of primes for which $f(x)$ has a particular factorisation pattern is equal to the proportion of elements in the Galois group $G(f)$ that have that same cycle structure.

For example, if the Galois group is the symmetric group $S_4$ of order $24$ and it contains six $4$-cycles, then we expect that for approximately $6/24 = 1/4$ of primes, the polynomial will be irreducible modulo $p$. Similarly, the density of primes for which the polynomial splits completely into linear factors corresponds to the proportion of the identity element in the group, which is $1/|G|$.

---

Conjecture: For a fixed irreducible polynomial $f$ with Galois group $G(f)$, the relative frequency of observing a particular cycle shape among the factorisations of $f$ modulo $p$ is proportional to the number of elements in $G(f)$ that have that same cycle structure as a permutation of the roots. Assuming the Galois group is the smallest possible, we infer it to be the smallest subgroup of the symmetric group $S_n$ that contains all the observed cycle types.

If a polynomial is reducible over the rational numbers $\mathbb{Q}$, its Galois group is not transitive. Instead, it is a subgroup of $S_n$ that acts separately on the roots of each irreducible factor. For the polynomials in this list that are irreducible over $\mathbb{Q}$, their Galois groups are transitive.



---

*   For $f(x) = x^2 + x + 41$, the observed cycle shapes are $(2)$ and $(1\;1)$.

    The only transitive subgroup of $S_2$ is $S_2$ itself, which is a cyclic group of order $2$. It consists of the identity and a transposition.
    $S_2$ has one element of each type. We predict that each cycle shape should appear for about 50% of primes, which is roughly in-line with our results.

*   For $f(x) = x^3 + x^2 - 2x - 1$, the observed cycle shapes are $(3)$ and $(1\;1\;1)$.

    The transitive subgroups of $S_3$ are the cyclic alternating group $A_3$ and $S_3$ itself. $S_3$ contains $2$-cycles, which were not observed. $A_3$ consists of the identity and two $3$-cycles, matching the results. Thus, the Galois group is $A_3$. We predict a $1 : 2$ ratio between the cycle shapes which is not matched exactly as we are only testing primes up to $p=97$.

*   For $f(x) = x^3 + 2x + 1$, the observed cycle shapes are $(3)$, $(1\;2)$ and $(1\;1\;1)$.

    Since this polynomial exhibits all possible cycle shapes for a degree $3$ polynomial, its Galois group must be the full symmetric group $S_3$. $S_3$ has $2$ elements of cycle type (3), $3$ elements of type (1\;2), and $1$ element of type (1\;1\;1). The observed frequencies are roughly in line with these proportions with completely splitting being quite rare.

*   For $f(x) = x^4 - 2x^2 + 4$, the observed cycle shapes are $(2\;2)$ and $(1\;1\;1\;1)$.

    This polynomial is reducible over $\mathbb{Q}$ via $(x^2 + 2x + 2)(x^2 - 2x + 2)$ so its Galois group is not transitive. The consistent cycle shape reflects the fact that it always splits into two quadratic factors and sometimes into linear factor. The Galois group is the Klein $4$-group $K_4$ which is a subgroup of S_4, and has $3$ element of type $(2\;2)$ and $1$ element of type $(1\;1\;1\;1)$.

*   For $f(x) = x^4 - 2x^3 + 5x + 5 $, the observed cycle shapes are $(4)$, $(2\;2)$, $(1\;3)$, $(1\;1\;2)$ and $(1\;1\;1\;1)$.
    
    The presence of a $4$-cycle and a $3$-cycle suggests that the Galois group is the full symmetric group $S_4$. This contains $6, 3, 8, 6$ elements of types $(4)$, $(2\;2)$, $(1\;3)$, $(1\;1\;2)$ respectively. We do not observe full splitting $(1\;1\;1\;1)$ because of our limited range of primes.

*   For $f(x) = x^4 - x^3 - 4x + 16$, the observed cycle shapes are $(4)$, $(2\;2)$, and $(1\;1\;2)$.

    The absence of a 3-cycle $(1\;3)$ suggests that the Galois group is not $S_4$ or $A_4$. The group must be a subgroup of $S_4$ that contains a $4$-cycle but no $3$-cycles which is $D_4$. This contains $2$ elements of cycle type $(4)$, $2$ elements of type $(1\;1\;2)$, $3$ elements of type $(2\;2)$ and $1$ element of type $(1\;1\;1\;1)$.

*   For $f(x) = x^4 + 7x^2 + 6x + 7 $, the observed cycle shapes are $(2\;2)$ and $(1\;1\;1\;1)$.
    
    This is reducible via $(x^2 + x + 1)(x^2 - x + 7)$ so its Galois group is not transitive. Here however the frequency of the cycle shapes $(2\;2)$ and $(1\;1\;1\;1)$ are roughly equal so we predict that the Galois group is in fact $C_2$.

We have fairly strong evidence in favor of the conjecture derived from the Chebotarev density theorem. The variety of cycle shapes that appear for a given polynomial is demonstrably useful for determining its Galois group by narrowing down possibilities. For polynomials that are reducible over the rationals, their cycle structures are constrained and reflect this reducibility.

In [83]:
quartics_polynomials_data = {
    "x^4 + x + 1": [1, 0, 0, 1, 1], # S_4 Resolvant cubic irreducible, discriminant not a square
    "x^4 + 8x + 12": [1, 0, 0, 8, 12], # A_4 Resolvant cubic irreducible, discriminant is a square
    "x^4 - 2": [1, 0, 0, 0, -2], # D_4 Resolvant cubic has one rational root
    "x^4 + 5x^2 + 5": [1, 0, 5, 0, 5], # C_4 Resolvant cubic has one rational root and f reducible adjoining discriminant
    "x^4 + 1": [1, 0, 0, 0, 1], # K_4 Resolvant cubic has three rational roots

    "(x^3 - x - 1)(x-5)": [1, -5, -1, 4, 5], # S_3
    "(x^3 - 3x - 1)(x-5)": [1, -5, -3, 14, 5], # C_3 or A_3
    "(x^2 - 1)(x^2 - 3)": [1, 0, -5, 0, 6], # K_4 Different splitting field
    "(x^2 + x + 1)(x^2 - x + 7)": [1, 0, 7, 6, 7], # C_2 Same splitting field
    "(x^2 - 2)(x-3)(x-4)": [1, -7, 10, 14, -24], # C_2
    "(x-1)(x-2)(x-3)(x-4)": [1, -10, 35, -50, 24], # I

    "x^3 - x - 1": [1, 0, -1, -1], # S_3 Discriminant not a square
    "x^3 - 3x - 1": [1, 0, -3, -1], # C_3 or A_3 Discriminant is a square

    "(x^2 - 2)(x-5)": [1, -5, -2, 10], # C_2
    "(x-1)(x-2)(x-3)": [1, -6, 11, -6], # I

    "x^2 - 2": [1, 0, -2], # S_2 = C_2
}
run_analysis_and_tabulate(quartics_polynomials_data, limit=101)


Decomposition Groups (Part 1)
  p   x^4 + x + 1    x^4 + 8x + 12    x^4 - 2    x^4 + 5x^2 + 5    x^4 + 1    (x^3 - x - 1)(x-5)    (x^3 - 3x - 1)(x-5)    (x^2 - 1)(x^2 - 3)
---  -------------  ---------------  ---------  ----------------  ---------  --------------------  ---------------------  --------------------
  2        4            Undef.        Undef.         Undef.        Undef.            1 3                    1 3                  Undef.
  3       1 3           Undef.          2^2            4             2^2             1 3                  Undef.                 Undef.
  5       1 3             1 3            4           Undef.          2^2            1^2 2                   1 3                   2^2
  7        4              1 3          1^2 2           4             2^2            Undef.                  1 3                  1^2 2
 11       1 3             1 3           2^2           1^4            2^2            1^2 2                   1 3                  1^2 2
 13     